## 2. Fasea: Algoritmoak diseinatzen

#### Julen Etxaniz eta Aitor Zubillaga

Community Detection proiektuaren 1. fasea entregatu duzue, eta feedback-a jaso ere. Klasean hainbat algoritmo ikusi ditugu, batzuk soluzio bakarrean oinarritutakoak, beste batzuk aldiz, populazio bat erabiltzen dutenak. Horiez gain, hibridatzeko teknikak ere ikusi ditugu. Bigarrengo fase honetan, bi algoritmo diseinatu beharko dituzue, bata soluzio bakarrean oinarritutakoa, eta bestea poblazionala. Ez hori bakarrik, bi algoritmoetako batek operadore probabilistikoak erabili beharko ditu, Estimation of Distribution Algorithms (EDAk) edo Ant Colony Optimization (ACO)-ek egiten duten bezala. Algoritmoen helburua, komunitate kopuru jakin bat emanik, modularitatea maximizatzen duen komunitate banaketa (soluzioa) bilatzen saiatzea da.

Errepasatu gaitegian zehar ikusi ditugun algoritmo guztiak, eta horiek kontuak izanik, libre zarete nahi dituzuen diseinuak sortzeko, baita ere hibridoak! Adi! Egiten duzuen aukeraketa argudiatu egin beharko duzue.


#### Entregablea

Bigarrengo fasea ebaluatu ahal izateko, notebook honetan bertan algoritmoen diseinua eta implementazioa proposatu beharko duzue. Gogoratu algoritmo bat azaltzeko modurik errezena diagrama bat egitea dela. Adi! Atal bakoitzean hartutako erabakiak eta garatutako metodoak egoki argudiatu beharko dituzue. Ez argudiatzeak edo lana garaiz ez entregatzeak penalizazioa jasoko dute ebaluagarria den proiektuaren zati honetan. eGelan zehazten dira notebook-a igotzeko <b>egun eta orduak</b>.

Momentuz, ez daukazue algoritmoen exekuzio eta konparaketak egin behar. Hirugarren fasean, esperimentazioaren inguruko baldintzak emango dizkizuet, eta, horrez gain, txostenaren idazketa burutu beharko duzue.

```python
## Hemendik aurrera, intuizioa da nagusi...

Nire aukeraketa, UMDA bat swap-LS batekin eta metodo eraikitzaile batekin.

Eta bestea, ILS bat, metodo eraikitzaile estokastiko batekin eta,
```



## Grafoa sortu

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Ingeniaritza Informatikoa/4. Maila/2. Lauhilekoa/BH/Proiektua - Community Detection

/content/drive/MyDrive/Ingeniaritza Informatikoa/4. Maila/2. Lauhilekoa/BH/Proiektua - Community Detection


In [5]:
# SQL
import sqlite3

# Pandas
import pandas as pd

# Graph
import community
import networkx as nx

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Combinations
import itertools

def sortu_grafoa():
    # Get data
    connect = sqlite3.connect('data/database.sqlite')
    query = """
    SELECT pa.paper_id, pa.author_id, a.name
    FROM paper_authors AS pa JOIN papers AS p ON pa.paper_id = p.id
    JOIN authors as a ON pa.author_id = a.id
    WHERE p.Year BETWEEN '2014' AND '2015'
    """
    df = pd.read_sql(query, connect)

    # Have a look at data
    df.head(10)

    # Initialize graph
    G = nx.Graph()

    # Transform
    # Autorearen IDa erabili beharrean erabili izena.
    for p, a in df.groupby('paper_id')['name']: 
        for u, v in itertools.combinations(a, 2):
            if G.has_edge(u, v):
                G[u][v]['weight'] +=1
            else:
                G.add_edge(u, v, weight=1)
                
    # Print graph size
    print('Autore kopurua grafoan:', G.number_of_nodes())
    print('Elkarlan kopurua grafoan:', G.number_of_edges())
    
    return G

def bistaratu_grafoa(G):
    plt.figure(figsize=(13, 9))
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, node_size = 20, node_color='0.75', label=True)
    nx.draw_networkx_edges(G, pos, alpha=0.5, width=1)
    plt.show()

    plt.axis('off')
    plt.show()

## Helburu-funtzioa

In [6]:
import numpy as np
# 50 komunitate ausaz
soluzioa_1 = np.random.randint(50, size=1843)
# 10 komunitate ausaz
soluzioa_2 = np.random.randint(10, size=1843)
print(list(soluzioa_1))
print(list(soluzioa_2))

[7, 7, 11, 14, 31, 38, 5, 0, 6, 16, 6, 29, 44, 43, 38, 30, 19, 3, 7, 46, 24, 47, 33, 11, 17, 11, 36, 29, 42, 40, 13, 21, 44, 41, 37, 21, 44, 36, 42, 47, 30, 11, 26, 24, 46, 29, 10, 40, 1, 49, 29, 24, 15, 25, 15, 27, 32, 31, 7, 11, 49, 19, 7, 4, 10, 11, 47, 15, 28, 19, 44, 28, 0, 0, 45, 35, 40, 43, 32, 20, 33, 33, 34, 16, 37, 31, 6, 3, 29, 15, 47, 20, 15, 45, 32, 36, 18, 49, 41, 24, 17, 37, 12, 6, 39, 23, 26, 7, 20, 26, 2, 17, 2, 23, 36, 22, 6, 40, 12, 25, 32, 11, 15, 32, 9, 49, 46, 19, 16, 19, 32, 15, 34, 34, 46, 46, 37, 29, 33, 38, 47, 39, 17, 0, 34, 7, 19, 46, 9, 32, 35, 25, 8, 24, 32, 11, 11, 21, 9, 44, 20, 15, 36, 7, 35, 19, 19, 33, 43, 21, 31, 46, 11, 8, 3, 2, 33, 10, 47, 25, 26, 9, 6, 28, 27, 19, 21, 43, 14, 15, 32, 36, 1, 40, 17, 5, 9, 24, 41, 14, 32, 15, 0, 31, 4, 25, 25, 25, 20, 45, 10, 11, 47, 46, 28, 16, 12, 34, 3, 13, 4, 28, 15, 42, 21, 3, 42, 22, 43, 12, 9, 49, 47, 18, 38, 16, 1, 34, 26, 42, 28, 11, 16, 13, 21, 41, 22, 27, 9, 3, 44, 25, 32, 28, 23, 29, 15, 16, 21, 9, 12, 4

In [7]:
from itertools import product
import community
import networkx as nx
import numpy as np
from collections import defaultdict

### Helburu-funtzioa
def modularitatea(G, partition, weight='weight'):
    weights = defaultdict(float)
    degrees = defaultdict(float)
    sum = G.size(weight=weight)
    modularity = 0

    for node in G:
        community = partition[node]
        degrees[community] += G.degree(node, weight=weight)
        for neighbour, data in G[node].items():
            if partition[neighbour] == community:
                if neighbour == node: # 4 nodes have self edges
                    weights[community] += data[weight] * 2
                else:
                    weights[community] += data[weight]

    for community in set(partition.values()):
        modularity += (weights[community] / (2 * sum)) - (degrees[community] / (2 * sum)) ** 2
    return modularity

### Dei orokorrak
G = sortu_grafoa()

## Gure inplementazioa
partition1 = dict(zip(G.nodes, soluzioa_1))
partition2 = dict(zip(G.nodes, soluzioa_2))

print("Gure helburu-funtzioa:")
print("1. soluzioaren modularitatea:", modularitatea(G, partition1))
print("2. soluzioaren modularitatea:", modularitatea(G, partition2))
print("Community moduluko modularity:")
print("1. soluzioaren modularitatea:", community.modularity(partition1, G))
print("2. soluzioaren modularitatea:", community.modularity(partition2, G))

Autore kopurua grafoan: 1843
Elkarlan kopurua grafoan: 3215
Gure helburu-funtzioa:
1. soluzioaren modularitatea: -0.0038977594319191135
2. soluzioaren modularitatea: 0.0017711664690531238
Community moduluko modularity:
1. soluzioaren modularitatea: -0.0038977594319191135
2. soluzioaren modularitatea: 0.0017711664690531238


## 1. LS

https://www.sciencedirect.com/science/article/pii/S0378437119301402?casa_token=8HvHlfHWvXkAAAAA:uhL8V5PBT2QMlSptsIvYJedrCHGRjeZBuvVx-oBYMwJDERz9n0Olp2nh6YVrcFMo3hlxJytWCWs

https://www.worldscientific.com/doi/abs/10.1142/S0217979220500137

## 2. EDA

https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=EDA+community+detection&btnG=

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7000750

https://ieeexplore.ieee.org/abstract/document/8711535?casa_token=mRcZrc_nHyAAAAAA:XFkC-HfhuOJ7wePqpFhDDcRQP9js3AkGEQ-yqYk9_DVWkLn3rCe1CLZdy8QH9-i7GWghAssK8PQ